In [ ]:
import json
import polyline as pl
import pandas as pd

## Load all data

In [ ]:
original_route = json.load(open("final_sfbayarea_filled/train_bus_ebike_mtv_ucb.filled.json"))
sm_reroute = json.load(open("final_sfbayarea_filled_reroutes/train_bus_ebike_mtv_ucb.filled.reroute.json"))

In [ ]:
combined_spec = json.load(open("final_sfbayarea/train_bus_ebike_mtv_ucb.json"))
combined_reroute = json.load(open("final_sfbayarea_filled_reroutes/train_bus_ebike_mtv_ucb.filled.reroute.json"))

## Check input spec

First, we verify that we have a trajectory defined for each leg in the spec, and that the leg contains a polyline. This is true by spot-checking, but this checks it programmatically

In [ ]:
def check_polyline_existence(t_or_l):
    trajectory_key_list = t_or_l.keys() - ["id", "name", "mode", "start_loc", "end_loc", "multiple_occupancy"]
    if len(trajectory_key_list) != 1:
        print("key list = %s" % trajectory_key_list)
        return False
    trajectory_key = next(iter(trajectory_key_list))
    polyline_exists = (trajectory_key == "polyline" or trajectory_key == "polylines")
    if not polyline_exists:
        if trajectory_key == 'waypoint_coords':
            wc = t_or_l["waypoint_coords"]
            if type(wc) == list:
                polyline_exists_list = ["polyline" in w for w in wc]
                polyline_exists = pd.Series(polyline_exists_list).all()
            else:
                polyline_exists = "polyline" in wc
        else:
            polyline_exists = False
    else:
        pass
    print("Trajectory is defined using %s, includes polyline %s" % (trajectory_key, polyline_exists))
    if not polyline_exists:
        print("======POLYLINE NOT FOUND for %s" % t_or_l["id"])
    return polyline_exists

In [ ]:
for t in combined_spec["evaluation_trips"]:
    if "legs" not in t:
        print("Checking unimodal trip %s" % t["id"])
        check_polyline_existence(t)
        continue
    for l in t["legs"]:
        print("Checking leg %s in trip %s" % (l["id"], t["id"]))
        check_polyline_existence(l)